<a href="https://colab.research.google.com/github/mafloan/TelecomChallengeAllura/blob/main/TelecomChallengeAllura1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nueva sección

In [12]:
!pip install pandas matplotlib requests

In [14]:
# Either run the command below or the one after it, but not both.
!python telecomx_pipeline.py --from raw
# Or local:
# !python telecomx_pipeline.py --from local --path ./TelecomX_Data.json

python3: can't open file '/content/telecomx_pipeline.py': [Errno 2] No such file or directory


In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet

# Configuración de rutas
OUTPUT_DIR = "reports"
CSV_PATH = "outputs/telecomx_clean.csv"
MD_PATH = os.path.join(OUTPUT_DIR, "EDA_TelecomX.md")
PDF_PATH = os.path.join(OUTPUT_DIR, "EDA_TelecomX.pdf")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Cargar dataset
try:
    df = pd.read_csv(CSV_PATH)
except FileNotFoundError:
    raise FileNotFoundError("No se encontró el archivo limpio en outputs/. Corre primero telecomx_pipeline.py")

# 2. Resumen de datos
n_rows, n_cols = df.shape
summary = df.info()
nulos = df.isnull().sum().to_dict()

# 3. Distribución de churn
if "churn" in df.columns:
    churn_counts = df["churn"].value_counts()
    plt.figure()
    churn_counts.plot(kind="bar")
    plt.title("Distribución de Churn")
    plt.ylabel("Cantidad")
    churn_img = os.path.join(OUTPUT_DIR, "churn_dist.png")
    plt.savefig(churn_img)
    plt.close()
else:
    churn_counts = None
    churn_img = None

# 4. Variables categóricas top
cat_plots = []
for col in df.select_dtypes(include=["object"]).columns[:3]:
    plt.figure()
    df[col].value_counts().head(5).plot(kind="bar")
    plt.title(f"Top categorías en {col}")
    img_path = os.path.join(OUTPUT_DIR, f"cat_{col}.png")
    plt.savefig(img_path)
    plt.close()
    cat_plots.append(img_path)

# 5. Variables numéricas
num_plots = []
for col in df.select_dtypes(include=["int64", "float64"]).columns[:2]:
    plt.figure()
    df[col].hist(bins=30)
    plt.title(f"Distribución de {col}")
    img_path = os.path.join(OUTPUT_DIR, f"num_{col}.png")
    plt.savefig(img_path)
    plt.close()
    num_plots.append(img_path)

# 6. Guardar Markdown
with open(MD_PATH, "w", encoding="utf-8") as f:
    f.write("# Informe EDA - TelecomX\n\n")
    f.write(f"Filas: {n_rows}, Columnas: {n_cols}\n\n")
    f.write("## Valores nulos por columna\n")
    for k, v in nulos.items():
        f.write(f"- {k}: {v}\n")
    if churn_counts is not None:
        f.write("\n## Distribución de Churn\n")
        f.write(str(churn_counts.to_dict()))

# 7. Guardar PDF
styles = getSampleStyleSheet()
elements = []
doc = SimpleDocTemplate(PDF_PATH, pagesize=A4)
elements.append(Paragraph("<b>Informe EDA - TelecomX</b>", styles['Title']))
elements.append(Spacer(1, 12))
elements.append(Paragraph(f"Filas: {n_rows}, Columnas: {n_cols}", styles['Normal']))

# Nulos
elements.append(Paragraph("<b>Valores nulos por columna:</b>", styles['Heading2']))
for k, v in nulos.items():
    elements.append(Paragraph(f"{k}: {v}", styles['Normal']))

# Imágenes
def add_image(path, width=400):
    if path and os.path.exists(path):
        elements.append(Spacer(1, 12))
        elements.append(Image(path, width, width*0.6))

if churn_img:
    add_image(churn_img)
for img in cat_plots + num_plots:
    add_image(img)

doc.build(elements)

print(f"✅ Reportes generados: {MD_PATH} y {PDF_PATH}")


FileNotFoundError: No se encontró el archivo limpio en outputs/. Corre primero telecomx_pipeline.py

In [13]:
!pip install reportlab